In [1]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("rmanluo/RoG-webqsp", cache_dir="/SSL_NAS/concrete/data/webqsp")

combined_dataset = concatenate_datasets([dataset['train'], dataset['validation'], dataset['test']])

data = combined_dataset[0]

with open('/home/bonbak/ULTRA/custom/train.txt', 'w') as f:
    for triple in data['graph']:
        triple_str = '\t'.join(triple) + '\n'
        f.write(triple_str)

relation_list = []
for triple in data['graph']:
    h,r,t = triple
    relation_list.append(r)
relation_list = list(set(relation_list))

src, tgt = data['a_entity'][0], data['q_entity'][0]

with open('/home/bonbak/ULTRA/custom/inference_graph.txt', 'w') as f:
    for rel in relation_list:
        triple = (src,rel,tgt)
        triple_str = '\t'.join(triple) + '\n'
        f.write(triple_str)

/home/bonbak/anaconda3/envs/ULTRA/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from ultra import util
import yaml
import easydict

def load_yaml(args):
    with open(args.file_path, 'r') as file:
        content = file.read()
        content = content.replace("{{ dataset }}", "\""+args.dataset+"\"")
        content = content.replace("{{ gpus }}", "\""+str(args.gpus)+"\"")
        content = content.replace("{{ ckpt }}", "\""+args.checkpoint+"\"")
        content = content.replace("{{ bpe }}", "\""+args.bpe+"\"")
        content = content.replace("{{ epochs }}", "\""+args.epochs+"\"")
        config = yaml.safe_load(content)
        config = easydict.EasyDict(config)
    return config

/home/bonbak/anaconda3/envs/ULTRA/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class args:
    seed=1024
    dataset = 'CustomTransductiveDataset'
    gpus = 0
    checkpoint = '/home/bonbak/ULTRA/ckpts/ultra_50g.pth'
    file_path = '/home/bonbak/ULTRA/config/transductive/inference.yaml'
    bpe = 'null'
    epochs = '0'

cfg = load_yaml(args)

In [3]:
import torch

torch.manual_seed(args.seed + util.get_rank())

task_name = cfg.task["name"]
dataset = util.build_dataset(cfg)
device = f"cuda:"+str(args.gpus)

train_data, valid_data, test_data = dataset[0], dataset[1], dataset[2]
train_data = train_data.to(device)
valid_data = valid_data.to(device)
test_data = test_data.to(device)

CustomTransductiveDataset dataset
#train: 9088, #valid: 335, #test: 335


In [4]:
from ultra.models import Ultra
from torch.utils import data as torch_data
from ultra import tasks

model = Ultra(
    rel_model_cfg=cfg.model.relation_model,
    entity_model_cfg=cfg.model.entity_model,
)

if "checkpoint" in cfg and cfg.checkpoint is not None:
    state = torch.load(cfg.checkpoint, map_location="cpu")
    model.load_state_dict(state["model"])

#model = pyg.compile(model, dynamic=True)
model = model.to(device)

world_size = util.get_world_size()
rank = util.get_rank()

test_triplets = torch.cat([test_data.target_edge_index, test_data.target_edge_type.unsqueeze(0)]).t()
sampler = torch_data.DistributedSampler(test_triplets, world_size, rank)
test_loader = torch_data.DataLoader(test_triplets, test_data.num_edge_types, sampler=sampler)

In [5]:
# def load_vocab():
with open(f'{dataset.raw_dir}/entity.dict', 'r') as f:
    entities = f.readlines()

entity_dict = {}
for res in entities:
    id, ent = res[:-1].split('\t')
    entity_dict[int(id)] = ent


with open(f'{dataset.raw_dir}/relation.dict', 'r') as f:
    relations = f.readlines()

relation_dict = {}
for res in relations:
    id, rel = res[:-1].split('\t')
    relation_dict[int(id)] = rel

In [6]:
model.eval()
for batch in test_loader:
    triples = batch.unsqueeze(1)
    pred = model(train_data, triples)
    values, indices = pred.squeeze().topk(k=1)
    paths, weights = model.visualize(train_data, batch[indices].unsqueeze(0))

    inv_pred = model(train_data, triples[:, :, [1, 0, 2]])
    inv_values, inv_indices = inv_pred.squeeze().topk(k=1)
    inv_paths, inv_weights = model.visualize(train_data, batch[inv_indices].unsqueeze(0))

Load rspmm extension. This may take a while...


In [10]:
num_relation = len(relation_dict)
for path in paths:
    for triple in path:
        h,t,r = triple
        h_name, t_name = entity_dict[h], entity_dict[t]
        r_name = relation_dict[r % num_relation]
        if r >= num_relation:
            r_name += "^(-1)"

299 129 560
Jaxon Bieber people.person.sibling_s m.0gxnnwp
560 464 15


KeyError: 464

In [11]:
result_triple_list = batch[indices].detach().tolist()

for triple in result_triple_list:
    # dataset.inv_rel
    h,t,r = triple
    print(entity_dict[h], relation_dict[r], entity_dict[t])

result_triple_list = batch[inv_indices].detach().tolist()

for triple in result_triple_list:
    h,t,r = triple
    print(entity_dict[t], relation_dict[r],entity_dict[h])

Jaxon Bieber award.award_honor.award_winner Justin Bieber
Justin Bieber people.sibling_relationship.sibling Jaxon Bieber
